This cell prepares for an API call to the fabdb.net website to retrieve trading card data for the "Flesh and Blood TCG" game. It handles authentication by creating a secure hash and constructs the URL with necessary parameters and headers. Once this setup is complete, the program is ready to make the API call and fetch the card data from the website. I set the parameters to do the max per page and total pages of 19 so I could pull all of the cards with one request. 

In [1]:
import requests
import hashlib
import time
import pandas as pd
from pandas import json_normalize

secret_token = '696a6f47212d93f88cef71238cfb3d1cd357905b3e34eabd2bbc49124553c129'
# creating unix timestamp
timestamp = int(time.time())

#making query string for api call
query_string = f'time={timestamp}'

# Generate the hash
hash_string = secret_token + str(timestamp)
hash_value = hashlib.sha512(hash_string.encode()).hexdigest()

# url with hash and time variables added 
url = f'https://api.fabdb.net/cards?{query_string}&hash={hash_value}'

parameters = {
    "page": 1,
    "per_page": 100
}

total_pages = 19

# Set the request headers
headers = {
    'Authorization': f'Bearer {secret_token}',
    'Accept': 'application/json'
}


This cell then makes a list of the original data pulled from the first request and then paginates(loops) through the rest of the pages to collect all of the data and then use json.normalize to turn the collected data into a pandas dataframe. 

In [2]:
# making a list to store the data from the api response in to then loop through the pages for call card data
data_records = []

for page in range(1, total_pages + 1):
    parameters['page'] = page
    
    response = requests.get(url, params=parameters)

    # process response
    data = response.json()

    # Extract the relevant data records from the response
    data_records.extend(data['data'])
pd.set_option('display.max_columns', None)
# Normalize the data before storing it in the dataframe
df2 = pd.json_normalize(data_records)
df2.head(2)

,identifier,name,legality,keywords,text,rarity,image,printings,stats.resource,stats.life,stats.intellect,stats.attack,stats.defense,stats.cost,stats
0,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 863, 'language': 'en', 'name': 'Eye of...",3.0,NaN,NaN,NaN,NaN,NaN,NaN
1,dash-inventor-extraordinaire,"Dash, Inventor Extraordinaire",[],"[mechanologist, hero]",You may start the game with a Mechanologist it...,T,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 864, 'language': 'en', 'name': 'Dash, ...",NaN,40.0,4.0,NaN,NaN,NaN,NaN


In this cell i am exploding the printings column because it was still nested json. I am then taking those columns that explode out and making new columns for a wider data set while dropping the original printings column.

In [3]:
# Explode the 'printings' column to expand the list
df_printings = df2.explode('printings')

# Expand the 'printings' column into separate columns
df_printings = pd.concat([df_printings['printings'].apply(pd.Series), df_printings], axis=1)

# Merge the expanded data with the original DataFrame
df_merged = pd.concat([df2.drop('printings', axis=1), df_printings], axis=1)

# Drop the original 'printings' column
df_merged.drop('printings', axis=1, inplace=True)
df_merged.head(2)

,identifier,name,legality,keywords,text,rarity,image,stats.resource,stats.life,stats.intellect,stats.attack,stats.defense,stats.cost,stats,id,language,name,text,flavour,sku,set,rarity,finish,edition,image,identifier,name,legality,keywords,text,rarity,image,stats.resource,stats.life,stats.intellect,stats.attack,stats.defense,stats.cost,stats
0,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,863,en,Eye of Ophidia,**Legendary** *(You may only have 1 Eye of Oph...,"Beyond the turbulent waters of Death’s Knell, ...","{'sku': 'ARC000-CF', 'finish': 'cold', 'set': ...",Arcane Rising,F,Cold foil,{},https://fabdb2.imgix.net/cards/printings/ARC00...,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN
0,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1281,en,Eye of Ophidia,**Legendary** *(You may only have 1 Eye of Oph...,"Beyond the turbulent waters of Death’s Knell, ...","{'sku': 'U-ARC000-RF', 'finish': 'rainbow', 's...",Arcane Rising Unlimited,F,Rainbow foil,{},https://fabdb2.imgix.net/cards/printings/U-ARC...,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN


Here I use json.normalize again to break down the sku column as it didn't explode out and because of the data types of the objects inside the sku column couldn't be exploded so I searched for other means to do the same thing. This increased my dataset a lot and made the columns I needed in order to merge this dataset with the web scraped data later. 

In [4]:
# Use json_normalize to unnest the 'sku' column
df_sku = pd.json_normalize(df_merged['sku'])

# Reset the indexes of the DataFrames
df_merged.reset_index(drop=True, inplace=True)
df_sku.reset_index(drop=True, inplace=True)

# Combine the expanded 'sku' columns with the rest of the DataFrame
df_unnested = pd.concat([df_merged.drop('sku', axis=1), df_sku], axis=1)
df_unnested.head(2)

,identifier,name,legality,keywords,text,rarity,image,stats.resource,stats.life,stats.intellect,stats.attack,stats.defense,stats.cost,stats,id,language,name,text,flavour,set,rarity,finish,edition,image,identifier,name,legality,keywords,text,rarity,image,stats.resource,stats.life,stats.intellect,stats.attack,stats.defense,stats.cost,stats,sku,finish,number,set.id,set.name,set.released,set.browseable,set.draftable,set.0
0,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,863,en,Eye of Ophidia,**Legendary** *(You may only have 1 Eye of Oph...,"Beyond the turbulent waters of Death’s Knell, ...",Arcane Rising,F,Cold foil,{},https://fabdb2.imgix.net/cards/printings/ARC00...,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,ARC000-CF,cold,000,arc,Arcane Rising,2020-03-27,True,True,NaN
1,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1281,en,Eye of Ophidia,**Legendary** *(You may only have 1 Eye of Oph...,"Beyond the turbulent waters of Death’s Knell, ...",Arcane Rising Unlimited,F,Rainbow foil,{},https://fabdb2.imgix.net/cards/printings/U-ARC...,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,U-ARC000-RF,rainbow,000,arc,Arcane Rising,2020-03-27,True,True,NaN


I rename a lot of columns so that they make more sense here and are more similar to columns in the other data. doing this gave me more options on where I could merge the data and made the columns more formatted and neat. 

In [5]:
mapping = {
    'stats.resource': 'pitch value',
    'stats.life': 'life',
    'stats.attack': 'attack',
    'stats.defense': 'defense',
    'stats.cost': 'cost',
    'stats.intellect': 'intellect'
}

# Apply the mapping function to rename the 'stats' columns
df_unnested.rename(columns=mapping, inplace=True)

# Display the DataFrame with updated column names
df_unnested.head(2)

,identifier,name,legality,keywords,text,rarity,image,pitch value,life,intellect,attack,defense,cost,stats,id,language,name,text,flavour,set,rarity,finish,edition,image,identifier,name,legality,keywords,text,rarity,image,pitch value,life,intellect,attack,defense,cost,stats,sku,finish,number,set.id,set.name,set.released,set.browseable,set.draftable,set.0
0,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,863,en,Eye of Ophidia,**Legendary** *(You may only have 1 Eye of Oph...,"Beyond the turbulent waters of Death’s Knell, ...",Arcane Rising,F,Cold foil,{},https://fabdb2.imgix.net/cards/printings/ARC00...,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,ARC000-CF,cold,000,arc,Arcane Rising,2020-03-27,True,True,NaN
1,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1281,en,Eye of Ophidia,**Legendary** *(You may only have 1 Eye of Oph...,"Beyond the turbulent waters of Death’s Knell, ...",Arcane Rising Unlimited,F,Rainbow foil,{},https://fabdb2.imgix.net/cards/printings/U-ARC...,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,U-ARC000-RF,rainbow,000,arc,Arcane Rising,2020-03-27,True,True,NaN


Here i am making a dictionary for the rarity column and then replacing the values in the rows of that column replacing the single letters to word, matching the web scraped so I could merge on this column. 

In [6]:
# Define the mapping dictionary for rarity
rarity_mapping = {
    'F': 'Fabled',
    'M': 'Majestic',
    'C': 'Common',
    'R': 'Rare',
    'L': 'Legendary'
}

# Use the replace method to update the 'rarity' column
df_unnested['rarity'] = df_unnested['rarity'].replace(rarity_mapping)

# Display the DataFrame with updated rarity values
df_unnested.head(2)

,identifier,name,legality,keywords,text,rarity,image,pitch value,life,intellect,attack,defense,cost,stats,id,language,name,text,flavour,set,rarity,finish,edition,image,identifier,name,legality,keywords,text,rarity,image,pitch value,life,intellect,attack,defense,cost,stats,sku,finish,number,set.id,set.name,set.released,set.browseable,set.draftable,set.0
0,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,Fabled,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,863,en,Eye of Ophidia,**Legendary** *(You may only have 1 Eye of Oph...,"Beyond the turbulent waters of Death’s Knell, ...",Arcane Rising,Fabled,Cold foil,{},https://fabdb2.imgix.net/cards/printings/ARC00...,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,Fabled,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,ARC000-CF,cold,000,arc,Arcane Rising,2020-03-27,True,True,NaN
1,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,Fabled,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1281,en,Eye of Ophidia,**Legendary** *(You may only have 1 Eye of Oph...,"Beyond the turbulent waters of Death’s Knell, ...",Arcane Rising Unlimited,Fabled,Rainbow foil,{},https://fabdb2.imgix.net/cards/printings/U-ARC...,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,Fabled,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,U-ARC000-RF,rainbow,000,arc,Arcane Rising,2020-03-27,True,True,NaN


Here I am exporting this dataframe to a .csv so I can merge the data in the merged_data notebook. 

In [7]:
df_unique = df_unnested.loc[:,~df_unnested.columns.duplicated()].copy()
df_unique.head(2)


,identifier,name,legality,keywords,text,rarity,image,pitch value,life,intellect,attack,defense,cost,stats,id,language,flavour,set,finish,edition,sku,number,set.id,set.name,set.released,set.browseable,set.draftable,set.0
0,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,Fabled,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,863,en,"Beyond the turbulent waters of Death’s Knell, ...",Arcane Rising,Cold foil,{},ARC000-CF,000,arc,Arcane Rising,2020-03-27,True,True,NaN
1,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,Fabled,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1281,en,"Beyond the turbulent waters of Death’s Knell, ...",Arcane Rising Unlimited,Rainbow foil,{},U-ARC000-RF,000,arc,Arcane Rising,2020-03-27,True,True,NaN


In [8]:
df_unique.to_csv('API_data.csv', index=False)